# Ablation studies

In this notebook we give the scripts for various ablation studies resented in out paper:
- **Weights**<br>
    Verifying the role of the pair of constant shared weights used for `epsilon` computation.<br>
- **Batch size**<br>
    Investigating the effect of the batch size.<br>
- **Synthetic data**<br>
    Changing the input data to verify dependency of the method on the data.
- **Initializations**<br>
    Changing initialization scheme.

In [1]:
import os
import json
import time
import itertools

import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
from scipy import stats
from tqdm import trange
from dotmap import DotMap
from statistics import mean
import matplotlib.pyplot as plt
from prettytable import PrettyTable

import torch.nn as nn
import torch

import nasspace
from datasets import data
from epsilon_utils import prepare_seed, compute_stats, plot_results, search_weights

2023-01-17 10:02:59.570159: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2023-01-17 10:02:59.570280: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2023-01-17 10:02:59.570292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above

## Weight ablations

In this ablation study we verify how much does the performance of epsilon metric change depending on the chosen weights.

In [8]:
dataset = 'cifar10'
data_loc = './datasets/cifardata'
batch_size = 256
repeat = 1
GPU = '0'
augtype = 'none'
trainval = False

In [9]:
# Arguments required for NAS-Bench-201 search space initialisation
args = DotMap()

args.api_loc = './nasbench_only108.tfrecord'
args.nasspace = 'nasbench101'
args.dataset = dataset
args.stem_out_channels = 128
args.num_stacks = 3
args.num_modules_per_stack = 3
args.num_labels = 1

savedataset = dataset
dataset = 'fake' if 'fake' in savedataset else savedataset
savedataset = savedataset.replace('fake', '')
if savedataset == 'cifar10':
    savedataset = savedataset + '-valid'

In [10]:
# Load the search space (it takes some time)
searchspace = nasspace.get_search_space(args)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 63 seconds


In [11]:
if 'valid' in savedataset:
    savedataset = savedataset.replace('-valid', '')
    
if args.dataset == 'cifar10':
    acc_type = 'ori-test'
    val_acc_type = 'x-valid'
else:
    acc_type = 'x-test'
    val_acc_type = 'x-valid'

In [12]:
# Define the device
os.environ['CUDA_VISIBLE_DEVICES'] = GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Randomly select n_samples architectures
prepare_seed(21)

In [13]:
# Load the data 
train_loader = data.get_data(dataset, data_loc, trainval, batch_size, augtype, repeat, args)

# Pick up a batch
data_iterator = iter(train_loader)
x, _= next(data_iterator)
x = x.to(device)

0it [00:00, ?it/s]

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./datasets/cifardata/cifar-10-python.tar.gz


0it [00:00, ?it/s]

Extracting ./datasets/cifardata/cifar-10-python.tar.gz to ./datasets/cifardata
Files already downloaded and verified


In [3]:
weight_range = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
it = 0
for weight_comb in itertools.product(weight_range, weight_range):
    weight_l = weight_comb[0]
    weight_h = weight_comb[1]
    
    if weight_h>weight_l:
        save_dir = './release_results/NAS-Bench-101/Ablation/Weights/Weight_{}_{}/BS{}/'.format(weight_l, weight_h, args.batch_size)
        os.makedirs(save_dir, exist_ok=True)
        
        if os.path.exists(save_dir+'Data'):
            # Load precomputed results
            data_file = open(save_dir+'Data','rb')
            input_data = pkl.load(data_file)
            score = input_data["score"]
            accs_mean = input_data["accs_mean"]
            accs_min = input_data["accs_min"]
            accs_max = input_data["accs_max"]
            nparams = input_data["nparams"]
        else:
            accs_mean = []
            accs_min = []
            accs_max = []
            nparams = []
            score = []
            for i in trange(5000):
                uid = searchspace[i]
                network = searchspace.get_network(uid)
                network = network.to(device)
                preds = []
                for weight in [weight_l, weight_h]:
                    
                    torch.cuda.empty_cache()
                    prepare_seed(21)
                    
                    # Initialize
                    def initialize_resnet(m):
                        if type(m) == torch.nn.Sequential:
                            for sub_m in m:
                                initialize_resnet(sub_m)
                        else:
                            fill_bias = False
                            if hasattr(m, 'bias'):
                                if m.bias is not None:
                                    fill_bias = True

                            if fill_bias:
                                torch.nn.init.constant_(m.bias, 0)

                            fill_weight = False
                            if hasattr(m, 'weight'):
                                fill_weight = True

                            if hasattr(m, 'affine'):
                                if not m.affine:
                                    fill_weight = False

                            if fill_weight:
                                torch.nn.init.constant_(m.weight, weight)

                    network.apply(initialize_resnet)
                    y_pred, _ = network(x)
                    pred = y_pred.cpu().detach().numpy().flatten()
                    pred_min = np.nanmin(pred)
                    pred_max = np.nanmax(pred)
                    pred_norm = (pred - pred_min)/(pred_max - pred_min)
                    preds.append(pred_norm)

                # Compute the score
                preds = np.array(preds)
                preds[np.where(preds==0)] = np.nan
                mae = np.nanmean(np.abs(preds[0,:]-preds[1,:]))
                mean = np.nanmean(preds)

                score.append(mae/mean)
                nparams.append(sum(p.numel() for p in network.parameters()))
                accs_mean.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[0])
                accs_min.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[1])
                accs_max.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[2]) 

            save_dic = {}
            save_dic["score"] = score
            save_dic["accs_mean"] = accs_mean
            save_dic["accs_min"] = accs_min
            save_dic["accs_max"] = accs_max
            save_dic["nparams"] = nparams
            pkl.dump(save_dic, open(save_dir + "Data", "wb"))
            
        if it==0:
            # Make the table
            headers = ["Weights", "Archs", "Spearman (global)", "Spearman (top-10%)", "Kendall (global)", "Kendall (top-10%)", "Top-10%/top-10%", "Top-64/top-5%"]
            table = PrettyTable(headers)
        stats, remain = compute_stats(score, accs_mean)
        stats_print = ["[{}, {}]".format(weight_l, weight_h), remain] + stats     
        table.add_row(stats_print)
        it+=1
        
#         plot_results(score=score,
#                      accs=accs_mean,
#                      nparams=nparams,
#                      top10=False,
#                      log_scale=False,
#                      save_dir=save_dir,
#                      save_name="Epsilon_{}_{}.png".format(weight_l, weight_h))

In [ ]:
weights = search_weights(x, searchspace, device)

In [4]:
import pandas as pd
pd.DataFrame(table.rows)

,0,1,2,3,4,5,6,7
0,"[0.0001, 0.001]",2120,0.47,0.41,0.33,0.28,38.21,23.00
1,"[0.0001, 0.01]",2120,0.61,0.03,0.43,0.02,41.98,24.00
2,"[0.0001, 0.1]",2120,0.61,0.03,0.43,0.02,41.98,24.00
3,"[0.0001, 1]",2120,0.61,0.03,0.43,0.02,41.98,24.00
4,"[0.0001, 10]",2120,0.61,0.03,0.43,0.02,41.98,24.00
5,"[0.001, 0.01]",4968,0.29,-0.05,0.20,-0.03,17.30,10.00
6,"[0.001, 0.1]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
7,"[0.001, 1]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
8,"[0.001, 10]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
9,"[0.01, 0.1]",5000,0.21,-0.12,0.14,-0.08,4.60,0.00


In [10]:
import pandas as pd
pd.DataFrame(table.rows)

,0,1,2,3,4,5,6,7
0,"[0.0001, 0.001]",2120,0.47,0.41,0.33,0.28,38.21,23.00
1,"[0.0001, 0.01]",2120,0.61,0.03,0.43,0.02,41.98,24.00
2,"[0.0001, 0.1]",2120,0.61,0.03,0.43,0.02,41.98,24.00
3,"[0.0001, 1]",2120,0.61,0.03,0.43,0.02,41.98,24.00
4,"[0.0001, 10]",2120,0.61,0.03,0.43,0.02,41.98,24.00
5,"[0.001, 0.01]",4968,0.29,-0.05,0.20,-0.03,17.30,10.00
6,"[0.001, 0.1]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
7,"[0.001, 1]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
8,"[0.001, 10]",4968,0.29,-0.05,0.19,-0.03,17.30,8.00
9,"[0.01, 0.1]",5000,0.21,-0.12,0.14,-0.08,4.60,0.00


## Batch size

In [ ]:
batch_size_range = [256, 1024]

weight_l = 0.001
weight_h = 10

batch_all_spearman_all = []
batch_top10_spearman_all = []
batch_all_kendall_all = []
batch_top10_kendall_all = []
batch_top10top10_all = []
batch_top64top5_all = []

for batch_size in batch_size_range:
    batch_all_spearman = []
    batch_top10_spearman = []
    batch_all_kendall = []
    batch_top10_kendall = []
    batch_top10top10 = []
    batch_top64top5 = []
    
    for it in range(10):
        # Load the data batch
        if it==0:
            train_loader = data.get_data(args.dataset, args.data_loc, args.trainval, batch_size, args.augtype, args.repeat, args)
        data_iterator = iter(train_loader)
        x, _= next(data_iterator)
        x = x.to(device)

        save_dir = './release_results/NAS-Bench-101/Ablation/Batch Size/BS{}/'.format(batch_size)
        os.makedirs(save_dir, exist_ok=True)

        if os.path.exists(save_dir+'Data_' + str(it)):
            # Load precomputed results
            data_file = open(save_dir + 'Data_' + str(it),'rb')
            input_data = pkl.load(data_file)
            score = input_data["score"]
            accs_mean = input_data["accs_mean"]
            accs_min = input_data["accs_min"]
            accs_max = input_data["accs_max"]
            nparams = input_data["nparams"]
        else:
            accs_mean = []
            accs_min = []
            accs_max = []
            nparams = []
            score = []
            for i in trange(5000):
                uid = searchspace[i]
                network = searchspace.get_network(uid)
                network = network.to(device)
                preds = []
                for weight in [weight_l, weight_h]:

                    torch.cuda.empty_cache()
                    prepare_seed(21)

                    # Initialize
                    def initialize_resnet(m):
                        if type(m) == torch.nn.Sequential:
                            for sub_m in m:
                                initialize_resnet(sub_m)
                        else:
                            fill_bias = False
                            if hasattr(m, 'bias'):
                                if m.bias is not None:
                                    fill_bias = True

                            if fill_bias:
                                torch.nn.init.constant_(m.bias, 0)

                            fill_weight = False
                            if hasattr(m, 'weight'):
                                fill_weight = True

                            if hasattr(m, 'affine'):
                                if not m.affine:
                                    fill_weight = False

                            if fill_weight:
                                torch.nn.init.constant_(m.weight, weight)

                    network.apply(initialize_resnet)
                    y_pred, _ = network(x)
                    pred = y_pred.cpu().detach().numpy().flatten()
                    pred_min = np.nanmin(pred)
                    pred_max = np.nanmax(pred)
                    pred_norm = (pred - pred_min)/(pred_max - pred_min)
                    preds.append(pred_norm)

                # Compute the score
                preds = np.array(preds)
                preds[np.where(preds==0)] = np.nan
                mae = np.nanmean(np.abs(preds[0,:]-preds[1,:]))
                mean = np.nanmean(preds)

                score.append(mae/mean)
                nparams.append(sum(p.numel() for p in network.parameters()))
                accs_mean.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[0])
                accs_min.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[1])
                accs_max.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[2])

            save_dic = {}
            save_dic["score"] = score
            save_dic["accs_mean"] = accs_mean
            save_dic["accs_min"] = accs_min
            save_dic["accs_max"] = accs_max
            save_dic["nparams"] = nparams
            pkl.dump(save_dic, open(save_dir + "Data_" + str(it), "wb"))

        stats, remain = compute_stats(score, accs_mean, raw=True)
        batch_all_spearman.append(stats[0])
        batch_top10_spearman.append(stats[1])
        batch_all_kendall.append(stats[2])
        batch_top10_kendall.append(stats[3])
        batch_top10top10.append(stats[4])
        batch_top64top5.append(stats[5])

    batch_all_spearman_all.append(batch_all_spearman)
    batch_top10_spearman_all.append(batch_top10_spearman)
    batch_all_kendall_all.append(batch_all_kendall)
    batch_top10_kendall_all.append(batch_top10_kendall)
    batch_top10top10_all.append(batch_top10top10)
    batch_top64top5_all.append(batch_top64top5)

Files already downloaded and verified
Files already downloaded and verified


  0%|                                          | 1/5000 [00:00<24:54,  3.35it/s]/home/gracheva/miniconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide
/home/gracheva/miniconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Mean of empty slice
100%|███████████████████████████████████████| 5000/5000 [24:33<00:00,  3.39it/s]


Files already downloaded and verified
Files already downloaded and verified


 32%|████████████▍                          | 1593/5000 [19:44<30:49,  1.84it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  1%|▏                                      | 26/5000 [00:22<1:43:20,  1.25s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 70%|███████████████████████████▎           | 3499/5000 [44:18<12:39,  1.98it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

In [ ]:
batch_size_range = [8, 16, 32, 64, 128, 256, 512, 1024]

weight_l = 0.0001
weight_h = 10

batch_all_spearman_all = []
batch_top10_spearman_all = []
batch_all_kendall_all = []
batch_top10_kendall_all = []
batch_top10top10_all = []
batch_top64top5_all = []

for batch_size in batch_size_range:
    batch_all_spearman = []
    batch_top10_spearman = []
    batch_all_kendall = []
    batch_top10_kendall = []
    batch_top10top10 = []
    batch_top64top5 = []
    
    for it in range(10):
        # Load the data batch
        if it==0:
            train_loader = data.get_data(args.dataset, args.data_loc, args.trainval, batch_size, args.augtype, args.repeat, args)
        data_iterator = iter(train_loader)
        x, _= next(data_iterator)
        x = x.to(device)

        save_dir = './release_results/NAS-Bench-101/Ablation/Batch Size 1e-4 1/BS{}/'.format(batch_size)
        os.makedirs(save_dir, exist_ok=True)

        if os.path.exists(save_dir+'Data_' + str(it)):
            # Load precomputed results
            data_file = open(save_dir + 'Data_' + str(it),'rb')
            input_data = pkl.load(data_file)
            score = input_data["score"]
            accs_mean = input_data["accs_mean"]
            nparams = input_data["nparams"]
        else:
            accs_mean = []
            accs_min = []
            accs_max = []
            nparams = []
            score = []
            for i in trange(5000):
                uid = searchspace[i]
                network = searchspace.get_network(uid)
                network = network.to(device)
                preds = []
                for weight in [weight_l, weight_h]:

                    torch.cuda.empty_cache()
                    prepare_seed(21)

                    # Initialize
                    def initialize_resnet(m):
                        if type(m) == torch.nn.Sequential:
                            for sub_m in m:
                                initialize_resnet(sub_m)
                        else:
                            fill_bias = False
                            if hasattr(m, 'bias'):
                                if m.bias is not None:
                                    fill_bias = True

                            if fill_bias:
                                torch.nn.init.constant_(m.bias, 0)

                            fill_weight = False
                            if hasattr(m, 'weight'):
                                fill_weight = True

                            if hasattr(m, 'affine'):
                                if not m.affine:
                                    fill_weight = False

                            if fill_weight:
                                torch.nn.init.constant_(m.weight, weight)

                    network.apply(initialize_resnet)
                    y_pred, _ = network(x)
                    pred = y_pred.cpu().detach().numpy().flatten()
                    pred_min = np.nanmin(pred)
                    pred_max = np.nanmax(pred)
                    pred_norm = (pred - pred_min)/(pred_max - pred_min)
                    preds.append(pred_norm)

                # Compute the score
                preds = np.array(preds)
                preds[np.where(preds==0)] = np.nan
                mae = np.nanmean(np.abs(preds[0,:]-preds[1,:]))
                mean = np.nanmean(preds)

                score.append(mae/mean)
                nparams.append(sum(p.numel() for p in network.parameters()))
                accs_mean.append(searchspace.get_final_accuracy(uid, acc_type, args.trainval)[0])

            save_dic = {}
            save_dic["score"] = score
            save_dic["accs_mean"] = accs_mean
            save_dic["nparams"] = nparams
            pkl.dump(save_dic, open(save_dir + "Data_" + str(it), "wb"))

        stats, remain = compute_stats(score, accs_mean, raw=True)
        batch_all_spearman.append(stats[0])
        batch_top10_spearman.append(stats[1])
        batch_all_kendall.append(stats[2])
        batch_top10_kendall.append(stats[3])
        batch_top10top10.append(stats[4])
        batch_top64top5.append(stats[5])

    batch_all_spearman_all.append(batch_all_spearman)
    batch_top10_spearman_all.append(batch_top10_spearman)
    batch_all_kendall_all.append(batch_all_kendall)
    batch_top10_kendall_all.append(batch_top10_kendall)
    batch_top10top10_all.append(batch_top10top10)
    batch_top64top5_all.append(batch_top64top5)

In [ ]:
from matplotlib.ticker import MaxNLocator
def plot_experiment(exp_list, name, filename):
    fig = plt.figure(figsize=(7.2,4.45))
    ax = fig.add_subplot(111)
    def plot_exp(exp, label):
        exp = np.array(exp)
        q_75 = np.nanquantile(exp, .75, axis=1)
        q_25 = np.nanquantile(exp, .25, axis=1)
        mean = np.nanmedian(exp, axis=1)
        ax.plot(range(len(mean)), mean, label=label)
        ax.fill_between(range(len(mean)), np.nanmin(exp, axis=1), np.nanmax(exp, axis=1), alpha=0.1)
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
#         ax.set_xticklabels(['','8','16','32'])
        ax.set_xticklabels(['','8','16','32','64','128','256','512','1024'])
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(16)
    for exp,ename in exp_list:
        plot_exp(exp,ename)
    plt.grid()
    plt.xlabel('Batch size', fontsize=22)
    if name=='rho':
        plt.ylabel(r'Spearman $\rho$', fontsize=22)
    elif name=='tau':
        plt.ylabel(r'Kendall $\tau$', fontsize=18)
#     plt.ylim(0.3,1)
    plt.legend(loc=4, fontsize=20)
    plt.savefig(filename,
                bbox_inches='tight', 
                dpi=300,
                format='pdf')
    plt.show()

In [ ]:
save_dir = './release_results/NAS-Bench-201/IMAGENET16-120/Ablation/Batch Size/'
filename = save_dir+"BatchSize_Spearman_201_IMAGENET.eps"
plot_experiment([(batch_all_spearman_all,'Overall'), (batch_top10_spearman_all,'Top-10%')], 'rho', filename)

In [ ]:
save_dir = './release_results/NAS-Bench-201/IMAGENET16-120/Ablation/Batch Size/'
filename = save_dir+"BatchSize_Kendall_201_IMAGENET.eps"
plot_experiment([(batch_all_kendall_all,'Overall'), (batch_top10_kendall_all,'Top-10%')], 'tau', filename)